*This notebook is for my learning purpose so the model might be not optimal nor efficient.*

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


References:
* https://medium.com/@kenneth.ca95/a-guide-to-transfer-learning-with-keras-using-resnet50-a81a4a28084b
* https://www.kaggle.com/vincentsiow/chest-ct-scan-using-resnet101v2
* https://stackoverflow.com/a/61656540/10275039

First we need to import all important modules.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import PIL
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet import preprocess_input

In [ ]:
tf.__version__

'2.12.0'

In [ ]:
path = "/content/drive/MyDrive/Train"
for files in os.listdir(path):
    print(os.path.join(path,files))

/content/drive/MyDrive/Train/malignant
/content/drive/MyDrive/Train/normal
/content/drive/MyDrive/Train/benign


Then we set the path for this data.

In [ ]:
train_path = "/content/drive/MyDrive/Train"
test_path = "/content/drive/MyDrive/Test"

# Helper Functions
taken from: https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/10_Fine-Tuning.ipynb <br>
These helper functions will be useful for later computation.

In [ ]:
# Helper-function for joining a directory and list of filenames.
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

In [ ]:
# Helper-function for plotting images
def plot_images(images, cls_true, cls_pred=None, smooth=True):

    assert len(images) == len(cls_true)

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3, figsize=(15,15))

    # Adjust vertical spacing.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # There may be less than 9 images, ensure it doesn't crash.
        if i < len(images):
            # Plot image.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Name of the true class.
            cls_true_name = class_names[cls_true[i]]

            # Show true and predicted classes.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Name of the predicted class.
                cls_pred_name = class_names[cls_pred[i]]

                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)

            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel)

        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [ ]:
# Helper-function for printing confusion matrix

# Import a function from sklearn to calculate the confusion-matrix.
from sklearn.metrics import confusion_matrix

def print_confusion_matrix(cls_pred):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Get the confusion matrix using sklearn.
    cm = confusion_matrix(y_true=cls_test,  # True class for test-set.
                          y_pred=cls_pred)  # Predicted class.

    print("Confusion matrix:")

    # Print the confusion matrix as text.
    print(cm)

    # Print the class-names for easy reference.
    for i, class_name in enumerate(class_names):
        print("({0}) {1}".format(i, class_name))

In [ ]:
# Helper-function for plotting example errors
def plot_example_errors(cls_pred):
    # cls_pred is an array of the predicted class-number for
    # all images in the test-set.

    # Boolean array whether the predicted class is incorrect.
    incorrect = (cls_pred != cls_test)

    # Get the file-paths for images that were incorrectly classified.
    image_paths = np.array(image_paths_test)[incorrect]

    # Load the first 9 images.
    images = load_images(image_paths=image_paths[0:9])

    # Get the predicted classes for those images.
    cls_pred = cls_pred[incorrect]

    # Get the true classes for those images.
    cls_true = cls_test[incorrect]

    # Plot the 9 images we have loaded and their corresponding classes.
    # We have only loaded 9 images so there is no need to slice those again.
    plot_images(images=images,
                cls_true=cls_true[0:9],
                cls_pred=cls_pred[0:9])

In [ ]:
# Function for calculating the predicted classes of the entire test-set and calling
# the above function to plot a few examples of mis-classified images.
def example_errors():
    # The Keras data-generator for the test-set must be reset
    # before processing. This is because the generator will loop
    # infinitely and keep an internal index into the dataset.
    # So it might start in the middle of the test-set if we do
    # not reset it first. This makes it impossible to match the
    # predicted classes with the input images.
    # If we reset the generator, then it always starts at the
    # beginning so we know exactly which input-images were used.
    test_generator.reset()

    # Predict the classes for all images in the test-set.
    y_pred = model.predict(test_generator, steps=STEPS_TEST)

    # Convert the predicted classes from arrays to integers.
    cls_pred = np.argmax(y_pred,axis=1)

    # Plot examples of mis-classified images.
    plot_example_errors(cls_pred)

    # Print the confusion matrix.
    print_confusion_matrix(cls_pred)

In [ ]:
# Helper-function for loading images
def load_images(image_paths):
    # Load the images from disk.
    images = [plt.imread(path) for path in image_paths]

    # Convert to a numpy array and return it.
    return np.asarray(images)

# Working on ResNet50

In [ ]:
# Set some important constants here
IMAGE_SIZE = 350
N_CLASSES = 3
BATCH_SIZE = 16

Begin to create data generator to arrange the dataset to be used later. This can makes the work easier.

In [ ]:
# ImageDataGenerator is needed because the dataset has no many data.
# The data augmentation can be useful to generate many augmented images from a single image

# train_datagen = ImageDataGenerator(
#       rescale=1./255,
#       rotation_range=0.4,
#       width_shift_range=0.2,
#       height_shift_range=0.2,
#       shear_range=0.2,
#       zoom_range=0.2,
#       horizontal_flip=True,
#       vertical_flip=True,
#       fill_mode='nearest')
train_datagen = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                   class_mode = 'categorical')

# test_datagen = ImageDataGenerator(rescale = 1.0/255.0)
test_datagen = ImageDataGenerator(dtype='float32', preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (IMAGE_SIZE, IMAGE_SIZE),
                                                   class_mode = 'categorical')

Found 1263 images belonging to 3 classes.
Found 0 images belonging to 0 classes.


In [ ]:
# save some values to be used later

cls_train = train_generator.classes
cls_test = test_generator.classes
class_names = list(train_generator.class_indices.keys())
print(class_names)
num_classes = train_generator.num_classes
print("num classes:",num_classes)

['benign', 'malignant', 'normal']
num classes: 3


In [ ]:
image_paths_train = path_join(train_path, train_generator.filenames)
image_paths_test = path_join(test_path, test_generator.filenames)

In [ ]:
STEPS_TEST = test_generator.n / BATCH_SIZE
STEPS_TEST

0.0

We import the pre-Trained ResNet50 from tensorflow, and we exclude the top layer because we will use Transfer Learning. We use Average Pooling layer and use weights from ImageNet dataset.

In [ ]:
res_model = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3))

94765736/94765736 [==============================] - 1s 0us/step


Make sure to set all layers to be not trainable except the last convolution layer. We need the last convolution layer to be trained so it can adapt our data during learning process. We do this to make the computation more efficient since the layers is trained before in the base model.

In [ ]:
for layer in res_model.layers:
    if 'conv5' not in layer.name:
        layer.trainable = False

In [ ]:
# Check if all layers except conv5 layers are not trainable
for i, layer in enumerate(res_model.layers):
    print(i, layer.name, "-", layer.trainable)

0 input_1 - False
1 conv1_pad - False
2 conv1_conv - False
3 conv1_bn - False
4 conv1_relu - False
5 pool1_pad - False
6 pool1_pool - False
7 conv2_block1_1_conv - False
8 conv2_block1_1_bn - False
9 conv2_block1_1_relu - False
10 conv2_block1_2_conv - False
11 conv2_block1_2_bn - False
12 conv2_block1_2_relu - False
13 conv2_block1_0_conv - False
14 conv2_block1_3_conv - False
15 conv2_block1_0_bn - False
16 conv2_block1_3_bn - False
17 conv2_block1_add - False
18 conv2_block1_out - False
19 conv2_block2_1_conv - False
20 conv2_block2_1_bn - False
21 conv2_block2_1_relu - False
22 conv2_block2_2_conv - False
23 conv2_block2_2_bn - False
24 conv2_block2_2_relu - False
25 conv2_block2_3_conv - False
26 conv2_block2_3_bn - False
27 conv2_block2_add - False
28 conv2_block2_out - False
29 conv2_block3_1_conv - False
30 conv2_block3_1_bn - False
31 conv2_block3_1_relu - False
32 conv2_block3_2_conv - False
33 conv2_block3_2_bn - False
34 conv2_block3_2_relu - False
35 conv2_block3_3_conv - 

After that, we create new model to connect it later with pre-trained model. Since we excluded the top layers, we need to make the new layers to fit the classification task on this data. In this case, we only have 4 classes, so make a dense layer that can output 4 classes only using softmax activation function.

In [ ]:
model = Sequential()
model.add(res_model)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(N_CLASSES, activation='softmax'))

Compile the model then the model is ready to train

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 23,602,051
Trainable params: 14,986,243
Non-trainable params: 8,615,808
_________________________________________________________________


We set some callbacks method for evaluation purpose

In [ ]:
# checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/Dataset02',
#                              monitor='val_loss',
#                             mode='auto',
#                             save_best_only=True)
# early_stopping = EarlyStopping(verbose=1, patience=3)

In [ ]:
history = model.fit(train_generator,
                    steps_per_epoch = 100,
                    validation_data = test_generator,
                    validation_steps = 50,
                    epochs = 1,
                    verbose = 1)

 79/100 [======================>.......] - ETA: 1:26 - loss: 0.8114 - acc: 0.7878

100/100 [==============================] - 346s 3s/step - loss: 0.8114 - acc: 0.7878


Now we plot the line graph to see how the model performs on this data

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()

KeyError: ignored

In [ ]:
result = model.evaluate(test_generator, steps=STEPS_TEST)

The model seems pretty good for now because it can reach 70% accuracy. Still, the model can be improved in other way. Last step, let's see what are the examples that can make the model fail to predict the images.

In [ ]:
example_errors()

We can see from the confusion matrix that our model are often predicts an image wrong but can still achieve the accuracy about 70%. Perhaps we need to do another approach to fine-tune the model so it can perform better.

---